# 032: Exponential Smoothing - Holt-Winters Forecasting 📊

## Learning Objectives
- Master **Simple Exponential Smoothing (SES)** for level-only series
- Understand **Holt's Linear Trend** method (double exponential smoothing)
- Implement **Holt-Winters Seasonal** method (triple exponential smoothing)
- Apply **additive vs multiplicative seasonality** modeling
- Optimize **smoothing parameters** (α, β, γ)
- Forecast **production yield** with seasonal patterns

---

## 🔄 Exponential Smoothing Methods

```mermaid
graph TD
    A[Time Series] --> B{Has Trend?}
    B -->|No| C{Has Seasonality?}
    B -->|Yes| D{Has Seasonality?}
    
    C -->|No| E[Simple ES<br/>Level only]
    C -->|Yes| F[Seasonal Naive<br/>or Decomposition]
    
    D -->|No| G[Holt's Method<br/>Level + Trend]
    D -->|Yes| H[Holt-Winters<br/>Level + Trend + Seasonal]
    
    H --> I{Seasonal Type?}
    I -->|Constant Amplitude| J[Additive]
    I -->|Growing Amplitude| K[Multiplicative]
```

---

## 📊 Exponential Smoothing vs ARIMA

| **Aspect** | **Exponential Smoothing** | **ARIMA** |
|------------|--------------------------|-----------|
| **Approach** | Weighted averages (recent > old) | Regression on past values/errors |
| **Stationarity** | NOT required | Required (via differencing) |
| **Seasonality** | Built-in (Holt-Winters) | Requires SARIMA |
| **Interpretability** | High (α, β, γ parameters) | Moderate (AR/MA coefficients) |
| **Speed** | Very fast | Fast |
| **Best For** | Short-term forecasts, simple patterns | Complex autocorrelation structures |

---

## 🎯 Key Concepts

### 1. **Simple Exponential Smoothing (SES)**

For **level-only** series (no trend, no seasonality):

$$
\hat{y}_{t+1|t} = \alpha y_t + (1-\alpha) \hat{y}_{t|t-1}
$$

Where:
- $\alpha$ = smoothing parameter (0 < α < 1)
- High α (0.8): Recent data weighted heavily (responsive to changes)
- Low α (0.2): Historical average weighted heavily (smooth, stable)

**Equivalent form:**
$$
\hat{y}_{t+1|t} = \hat{y}_{t|t-1} + \alpha e_t
$$
Where $e_t = y_t - \hat{y}_{t|t-1}$ (forecast error)

---

### 2. **Holt's Linear Trend Method**

For **level + trend** series:

$$
\begin{align}
\text{Level: } \ell_t &= \alpha y_t + (1-\alpha)(\ell_{t-1} + b_{t-1}) \\
\text{Trend: } b_t &= \beta(\ell_t - \ell_{t-1}) + (1-\beta) b_{t-1} \\
\text{Forecast: } \hat{y}_{t+h|t} &= \ell_t + h b_t
\end{align}
$$

Where:
- $\ell_t$ = estimated level at time t
- $b_t$ = estimated trend at time t
- $\alpha$ = level smoothing (0 < α < 1)
- $\beta$ = trend smoothing (0 < β < 1)

---

### 3. **Holt-Winters Seasonal Method**

**Additive Seasonality** (constant seasonal amplitude):
$$
\begin{align}
\text{Level: } \ell_t &= \alpha(y_t - s_{t-m}) + (1-\alpha)(\ell_{t-1} + b_{t-1}) \\
\text{Trend: } b_t &= \beta(\ell_t - \ell_{t-1}) + (1-\beta) b_{t-1} \\
\text{Seasonal: } s_t &= \gamma(y_t - \ell_t) + (1-\gamma) s_{t-m} \\
\text{Forecast: } \hat{y}_{t+h|t} &= \ell_t + h b_t + s_{t+h-m}
\end{align}
$$

**Multiplicative Seasonality** (seasonal amplitude grows with level):
$$
\begin{align}
\text{Level: } \ell_t &= \alpha \frac{y_t}{s_{t-m}} + (1-\alpha)(\ell_{t-1} + b_{t-1}) \\
\text{Trend: } b_t &= \beta(\ell_t - \ell_{t-1}) + (1-\beta) b_{t-1} \\
\text{Seasonal: } s_t &= \gamma \frac{y_t}{\ell_t} + (1-\gamma) s_{t-m} \\
\text{Forecast: } \hat{y}_{t+h|t} &= (\ell_t + h b_t) \times s_{t+h-m}
\end{align}
$$

Where:
- $m$ = seasonal period (12 for monthly, 4 for quarterly)
- $\gamma$ = seasonal smoothing (0 < γ < 1)

---

## 🔬 Post-Silicon Validation Application

### **Production Yield Forecasting**
- **Problem:** Weekly yield has trend (improving) + seasonal pattern (quarterly PM drops)
- **Holt-Winters Solution**: Multiplicative seasonality captures PM impact (proportional to yield level)
- **Business Value**: $3M+ savings via accurate capacity planning (3-month horizon)

---

### 📝 What's Happening in This Code?

**Purpose:** Import libraries for exponential smoothing implementation and visualization

**Key Points:**
- **statsmodels.tsa**: Contains `ExponentialSmoothing` class for Holt-Winters methods
- **simple_exp_smoothing**: Deprecated but useful for educational SES implementation
- **holtwinters**: Legacy interface for Holt-Winters (use ExponentialSmoothing instead)
- **matplotlib**: For time series visualization with forecast intervals

**Why This Matters:** Exponential smoothing requires NO stationarity assumption (unlike ARIMA), making it ideal for production yield forecasting with trend + seasonality.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

### 📝 Simple Exponential Smoothing (From Scratch)

**Purpose:** Implement SES from first principles to understand weighted averaging

**Key Points:**
- **Formula**: $\hat{y}_{t+1} = \alpha y_t + (1-\alpha) \hat{y}_t$ (recursive update)
- **α parameter**: Controls responsiveness (0.8 = follow recent data, 0.2 = smooth averaging)
- **Initialization**: Start with first observation ($\hat{y}_1 = y_1$)
- **No trend/seasonality**: SES assumes stationary level with random fluctuations

**Post-Silicon Example:** Daily ATE (Automated Test Equipment) utilization rate (level-only, no trend)

In [ ]:
class SimpleExponentialSmoothing:
    """Simple Exponential Smoothing (SES) - Level only"""
    
    def __init__(self, alpha=0.3):
        self.alpha = alpha  # Smoothing parameter
        self.fitted_values = []
    
    def fit(self, y):
        """Fit SES model: y_hat[t+1] = alpha * y[t] + (1-alpha) * y_hat[t]"""
        y = np.array(y)
        self.fitted_values = np.zeros_like(y)
        
        # Initialize with first observation
        self.fitted_values[0] = y[0]
        
        # Recursive smoothing
        for t in range(1, len(y)):
            self.fitted_values[t] = self.alpha * y[t-1] + (1 - self.alpha) * self.fitted_values[t-1]
        
        self.level = self.fitted_values[-1]  # Final smoothed level
        return self
    
    def forecast(self, steps=1):
        """Forecast future values (flat line at final level)"""
        return np.full(steps, self.level)

# Generate synthetic ATE utilization data (70-85%, random fluctuations)
np.random.seed(42)
ate_utilization = 77 + 5 * np.random.randn(100)  # Mean 77%, std 5%

# Fit SES with different alpha values
ses_low = SimpleExponentialSmoothing(alpha=0.2).fit(ate_utilization)
ses_high = SimpleExponentialSmoothing(alpha=0.8).fit(ate_utilization)

# Visualize
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Low alpha (smooth)
axes[0].plot(ate_utilization, label='Actual ATE Utilization %', alpha=0.5, marker='.')
axes[0].plot(ses_low.fitted_values, label='SES (α=0.2, smooth)', linewidth=2)
axes[0].set_title('Low α: Smooth Averaging (Historical Focus)')
axes[0].set_xlabel('Day')
axes[0].set_ylabel('Utilization %')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# High alpha (responsive)
axes[1].plot(ate_utilization, label='Actual ATE Utilization %', alpha=0.5, marker='.')
axes[1].plot(ses_high.fitted_values, label='SES (α=0.8, responsive)', linewidth=2, color='red')
axes[1].set_title('High α: Responsive to Recent Changes')
axes[1].set_xlabel('Day')
axes[1].set_ylabel('Utilization %')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"α=0.2 Final Level: {ses_low.level:.2f}%")
print(f"α=0.8 Final Level: {ses_high.level:.2f}%")
print(f"α=0.2 Forecast (next 5 days): {ses_low.forecast(5)}")
print(f"α=0.8 Forecast (next 5 days): {ses_high.forecast(5)}")

### 📝 Holt's Linear Trend Method (Double Exponential Smoothing)

**Purpose:** Extend SES to handle trend (level + slope)

**Key Points:**
- **Two components**: Level ($\ell_t$) and trend ($b_t$), updated recursively
- **Two parameters**: α (level smoothing) and β (trend smoothing)
- **Forecast**: Linear projection ($\hat{y}_{t+h} = \ell_t + h \cdot b_t$)
- **Damped variant**: Multiply trend by damping factor φ ($\hat{y}_{t+h} = \ell_t + (\phi + \phi^2 + ... + \phi^h) b_t$)

**Post-Silicon Example:** Weekly yield improving linearly (80% → 95% over 52 weeks due to process maturity)

In [ ]:
# Generate synthetic yield data with linear trend (improving process)
np.random.seed(42)
weeks = np.arange(1, 53)  # 52 weeks
true_yield = 80 + 0.3 * weeks + 2 * np.random.randn(52)  # 80% → 95.6%, noise
yield_data = pd.Series(true_yield, index=pd.date_range('2023-01-01', periods=52, freq='W'))

# Fit Holt's method using statsmodels
holt_model = Holt(yield_data).fit(smoothing_level=0.3, smoothing_trend=0.1, optimized=False)

# Forecast next 12 weeks
holt_forecast = holt_model.forecast(steps=12)

# Visualize
plt.figure(figsize=(14, 6))
plt.plot(yield_data, label='Actual Yield %', marker='o', linestyle='-', alpha=0.7)
plt.plot(holt_model.fittedvalues, label="Holt's Fitted Values", linewidth=2, color='orange')
plt.plot(holt_forecast, label='12-Week Forecast', linestyle='--', marker='s', color='red', linewidth=2)
plt.axhline(95, color='green', linestyle=':', label='Target Yield (95%)')
plt.title("Holt's Linear Trend Method - Yield Improvement Forecasting", fontsize=14)
plt.xlabel('Week')
plt.ylabel('Yield %')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Final Level (ℓ): {holt_model.level:.2f}%")
print(f"Final Trend (b): {holt_model.trend:.2f}% per week")
print(f"Forecast (Week 53-64): {holt_forecast.values.round(2)}")
print(f"\nProjected Week 64 Yield: {holt_forecast.iloc[-1]:.2f}% (Target: 95%)")

### 📝 Holt-Winters Seasonal Method (Triple Exponential Smoothing)

**Purpose:** Handle trend + seasonality (additive or multiplicative)

**Key Points:**
- **Three components**: Level ($\ell_t$), trend ($b_t$), seasonal ($s_t$)
- **Three parameters**: α (level), β (trend), γ (seasonal)
- **Additive**: Seasonal effect added ($\hat{y} = \ell + hb + s$), constant amplitude
- **Multiplicative**: Seasonal effect multiplied ($\hat{y} = (\ell + hb) \times s$), growing amplitude
- **Seasonal period (m)**: 12 for monthly, 4 for quarterly, 7 for daily

**Post-Silicon Example:** Weekly yield with quarterly PM drops (multiplicative seasonality - drops larger at higher yield levels)

In [ ]:
# Generate synthetic yield data with trend + quarterly seasonality
np.random.seed(42)
weeks = np.arange(1, 105)  # 2 years (104 weeks)
trend = 80 + 0.15 * weeks  # Linear improvement
seasonal_pattern = np.tile([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -10], 9)[:104]  # PM drop every 12 weeks
seasonal_yield = trend + seasonal_pattern + 2 * np.random.randn(104)

yield_seasonal = pd.Series(seasonal_yield, index=pd.date_range('2023-01-01', periods=104, freq='W'))

# Fit Holt-Winters with multiplicative seasonality
hw_model = ExponentialSmoothing(
    yield_seasonal, 
    trend='add',           # Additive trend
    seasonal='mul',        # Multiplicative seasonality (PM impact grows with yield)
    seasonal_periods=12    # Quarterly pattern (12 weeks)
).fit()

# Forecast next 24 weeks (2 quarters)
hw_forecast = hw_model.forecast(steps=24)

# Visualize
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Full history + forecast
axes[0].plot(yield_seasonal, label='Actual Yield %', marker='o', linestyle='-', alpha=0.6)
axes[0].plot(hw_model.fittedvalues, label='Holt-Winters Fitted', linewidth=2, color='orange')
axes[0].plot(hw_forecast, label='24-Week Forecast', linestyle='--', marker='s', color='red', linewidth=2)
axes[0].axhline(95, color='green', linestyle=':', label='Target Yield (95%)')
axes[0].set_title('Holt-Winters Multiplicative - Yield with Quarterly PM Drops', fontsize=14)
axes[0].set_xlabel('Week')
axes[0].set_ylabel('Yield %')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Last 52 weeks + forecast (zoomed)
recent_actual = yield_seasonal[-52:]
recent_fitted = hw_model.fittedvalues[-52:]
axes[1].plot(recent_actual, label='Actual (Last Year)', marker='o', linestyle='-', alpha=0.6)
axes[1].plot(recent_fitted, label='Fitted', linewidth=2, color='orange')
axes[1].plot(hw_forecast, label='Forecast (Next 6 Months)', linestyle='--', marker='s', color='red', linewidth=2)
axes[1].axhline(95, color='green', linestyle=':', label='Target Yield')
axes[1].set_title('Last Year + 24-Week Forecast (Zoomed)', fontsize=14)
axes[1].set_xlabel('Week')
axes[1].set_ylabel('Yield %')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Evaluate on test set (last 12 weeks)
train_data = yield_seasonal[:-12]
test_data = yield_seasonal[-12:]

hw_train_model = ExponentialSmoothing(
    train_data, trend='add', seasonal='mul', seasonal_periods=12
).fit()
hw_test_forecast = hw_train_model.forecast(steps=12)

rmse = np.sqrt(mean_squared_error(test_data, hw_test_forecast))
mae = mean_absolute_error(test_data, hw_test_forecast)

print(f"Holt-Winters Smoothing Parameters:")
print(f"  α (level): {hw_model.params['smoothing_level']:.4f}")
print(f"  β (trend): {hw_model.params['smoothing_trend']:.4f}")
print(f"  γ (seasonal): {hw_model.params['smoothing_seasonal']:.4f}")
print(f"\nTest Set Performance (last 12 weeks):")
print(f"  RMSE: {rmse:.2f}%")
print(f"  MAE: {mae:.2f}%")
print(f"\nNext PM Drop Forecast (Week 12): {hw_forecast.iloc[11]:.2f}%")
print(f"Post-PM Recovery (Week 13): {hw_forecast.iloc[12]:.2f}%")

### 📝 Additive vs Multiplicative Seasonality Comparison

**Purpose:** Understand when to use additive vs multiplicative seasonality

**Key Points:**
- **Additive**: Seasonal fluctuations constant over time (±5% regardless of level)
- **Multiplicative**: Seasonal fluctuations proportional to level (10% drop from 90% = 9%, from 80% = 8%)
- **Visual test**: If seasonal amplitude grows with trend → use multiplicative
- **AIC/BIC selection**: Fit both models, compare information criteria (lower is better)

**Post-Silicon Example:** Wafer throughput with weekly patterns (additive) vs yield with PM drops (multiplicative)

In [ ]:
# Generate two types of seasonal data
np.random.seed(42)
time = np.arange(1, 73)  # 72 weeks

# Additive seasonality (constant amplitude)
additive_seasonal = np.tile([0, 2, 4, 2, 0, -2, -4, -2], 9)[:72]
additive_data = 50 + 0.2 * time + additive_seasonal + np.random.randn(72)

# Multiplicative seasonality (amplitude grows with level)
multiplicative_seasonal = np.tile([1.0, 1.03, 1.05, 1.03, 1.0, 0.97, 0.95, 0.97], 9)[:72]
multiplicative_data = (50 + 0.2 * time) * multiplicative_seasonal + np.random.randn(72)

# Convert to pandas series
additive_series = pd.Series(additive_data, index=pd.date_range('2023-01-01', periods=72, freq='W'))
multiplicative_series = pd.Series(multiplicative_data, index=pd.date_range('2023-01-01', periods=72, freq='W'))

# Fit both model types to both datasets
results = {}

# Additive data - try both models
results['add_add'] = ExponentialSmoothing(additive_series, trend='add', seasonal='add', seasonal_periods=8).fit()
results['add_mul'] = ExponentialSmoothing(additive_series, trend='add', seasonal='mul', seasonal_periods=8).fit()

# Multiplicative data - try both models
results['mul_add'] = ExponentialSmoothing(multiplicative_series, trend='add', seasonal='add', seasonal_periods=8).fit()
results['mul_mul'] = ExponentialSmoothing(multiplicative_series, trend='add', seasonal='mul', seasonal_periods=8).fit()

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Additive data with additive model
axes[0, 0].plot(additive_series, label='Actual (Constant Amplitude)', alpha=0.6, marker='.')
axes[0, 0].plot(results['add_add'].fittedvalues, label='Additive Model', linewidth=2, color='green')
axes[0, 0].set_title(f'Additive Data → Additive Model (AIC: {results["add_add"].aic:.1f})')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Additive data with multiplicative model
axes[0, 1].plot(additive_series, label='Actual (Constant Amplitude)', alpha=0.6, marker='.')
axes[0, 1].plot(results['add_mul'].fittedvalues, label='Multiplicative Model', linewidth=2, color='orange')
axes[0, 1].set_title(f'Additive Data → Multiplicative Model (AIC: {results["add_mul"].aic:.1f})')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Multiplicative data with additive model
axes[1, 0].plot(multiplicative_series, label='Actual (Growing Amplitude)', alpha=0.6, marker='.')
axes[1, 0].plot(results['mul_add'].fittedvalues, label='Additive Model', linewidth=2, color='orange')
axes[1, 0].set_title(f'Multiplicative Data → Additive Model (AIC: {results["mul_add"].aic:.1f})')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Multiplicative data with multiplicative model
axes[1, 1].plot(multiplicative_series, label='Actual (Growing Amplitude)', alpha=0.6, marker='.')
axes[1, 1].plot(results['mul_mul'].fittedvalues, label='Multiplicative Model', linewidth=2, color='green')
axes[1, 1].set_title(f'Multiplicative Data → Multiplicative Model (AIC: {results["mul_mul"].aic:.1f})')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Model comparison table
print("Model Selection via AIC (lower is better):\n")
print(f"{'Data Type':<20} {'Model Type':<20} {'AIC':<10} {'Best?':<10}")
print("="*60)
print(f"{'Additive':<20} {'Additive':<20} {results['add_add'].aic:<10.2f} {'✅' if results['add_add'].aic < results['add_mul'].aic else ''}")
print(f"{'Additive':<20} {'Multiplicative':<20} {results['add_mul'].aic:<10.2f} {'✅' if results['add_mul'].aic < results['add_add'].aic else ''}")
print(f"{'Multiplicative':<20} {'Additive':<20} {results['mul_add'].aic:<10.2f} {'✅' if results['mul_add'].aic < results['mul_mul'].aic else ''}")
print(f"{'Multiplicative':<20} {'Multiplicative':<20} {results['mul_mul'].aic:<10.2f} {'✅' if results['mul_mul'].aic < results['mul_add'].aic else ''}")
print("\n✅ Lower AIC indicates better model fit")

## 📊 Exponential Smoothing Methods Comparison

Compare Simple, Double, and Triple Exponential Smoothing:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

# Generate synthetic time series
np.random.seed(42)
n = 100
dates = pd.date_range(start='2023-01-01', periods=n, freq='D')

# Components: trend + seasonality + noise
trend = np.linspace(50, 100, n)
seasonality = 10 * np.sin(2 * np.pi * np.arange(n) / 12)
noise = np.random.normal(0, 3, n)
data = trend + seasonality + noise

ts = pd.Series(data, index=dates)

# Split train/test
train_size = int(0.8 * n)
train, test = ts[:train_size], ts[train_size:]

# 1. Simple Exponential Smoothing (no trend, no seasonality)
ses_model = SimpleExpSmoothing(train).fit(smoothing_level=0.2)
ses_forecast = ses_model.forecast(len(test))

# 2. Double Exponential Smoothing / Holt's (trend, no seasonality)
des_model = ExponentialSmoothing(train, trend='add', seasonal=None).fit()
des_forecast = des_model.forecast(len(test))

# 3. Triple Exponential Smoothing / Holt-Winters (trend + seasonality)
tes_model = ExponentialSmoothing(train, trend='add', seasonal='add', seasonal_periods=12).fit()
tes_forecast = tes_model.forecast(len(test))

# Calculate errors
ses_mae = np.mean(np.abs(test - ses_forecast))
des_mae = np.mean(np.abs(test - des_forecast))
tes_mae = np.mean(np.abs(test - tes_forecast))

# Visualize
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Plot 1: All forecasts
ax1 = axes[0, 0]
ax1.plot(train.index, train, label='Train', color='blue', linewidth=2)
ax1.plot(test.index, test, label='Test (Actual)', color='black', linewidth=2, linestyle='--')
ax1.plot(test.index, ses_forecast, label=f'SES (MAE={ses_mae:.2f})', color='orange', linewidth=2, alpha=0.7)
ax1.plot(test.index, des_forecast, label=f'DES (MAE={des_mae:.2f})', color='green', linewidth=2, alpha=0.7)
ax1.plot(test.index, tes_forecast, label=f'TES (MAE={tes_mae:.2f})', color='red', linewidth=2, alpha=0.7)
ax1.axvline(x=train.index[-1], color='gray', linestyle=':', linewidth=2, label='Train/Test Split')
ax1.set_title('Exponential Smoothing Methods Comparison', fontsize=14, fontweight='bold')
ax1.set_xlabel('Date', fontsize=12)
ax1.set_ylabel('Value', fontsize=12)
ax1.legend(loc='upper left', fontsize=10)
ax1.grid(True, alpha=0.3)

# Plot 2: MAE comparison
ax2 = axes[0, 1]
methods = ['Simple\n(SES)', 'Double\n(DES/Holt)', 'Triple\n(TES/Holt-Winters)']
maes = [ses_mae, des_mae, tes_mae]
colors = ['orange', 'green', 'red']
bars = ax2.bar(methods, maes, color=colors, edgecolor='black', linewidth=2, alpha=0.7)
for bar, mae in zip(bars, maes):
    height = bar.get_height()
    ax2.text(bar.get_x() + bar.get_width()/2., height,
            f'{mae:.2f}', ha='center', va='bottom', fontsize=12, fontweight='bold')
ax2.set_ylabel('Mean Absolute Error', fontsize=12, fontweight='bold')
ax2.set_title('Forecast Accuracy (Lower is Better)', fontsize=14, fontweight='bold')
ax2.grid(axis='y', alpha=0.3)

# Plot 3: Method capabilities
ax3 = axes[1, 0]
capabilities = pd.DataFrame({
    'Method': ['SES', 'DES', 'TES'],
    'Handles Level': [1, 1, 1],
    'Handles Trend': [0, 1, 1],
    'Handles Seasonality': [0, 0, 1],
    'Complexity': [1, 2, 3]
})

x = np.arange(len(capabilities))
width = 0.2
ax3.bar(x - width*1.5, capabilities['Handles Level'], width, label='Level', color='blue', alpha=0.7, edgecolor='black')
ax3.bar(x - width/2, capabilities['Handles Trend'], width, label='Trend', color='green', alpha=0.7, edgecolor='black')
ax3.bar(x + width/2, capabilities['Handles Seasonality'], width, label='Seasonality', color='orange', alpha=0.7, edgecolor='black')
ax3.bar(x + width*1.5, capabilities['Complexity'], width, label='Complexity', color='red', alpha=0.7, edgecolor='black')

ax3.set_xticks(x)
ax3.set_xticklabels(capabilities['Method'])
ax3.set_ylabel('Capability (0=No, 1=Yes) / Complexity (1-3)', fontsize=11, fontweight='bold')
ax3.set_title('Method Capabilities', fontsize=14, fontweight='bold')
ax3.legend(loc='upper left', fontsize=10)
ax3.grid(axis='y', alpha=0.3)

# Plot 4: Method decision guide
ax4 = axes[1, 1]
ax4.axis('off')

decision_text = [
    "📋 When to Use Each Method:\n",
    "✅ Simple Exponential Smoothing (SES):",
    "   • Stationary data (no trend, no seasonality)",
    "   • Short-term forecasts",
    "   • Fast computation needed\n",
    "✅ Double Exponential Smoothing (DES/Holt):",
    "   • Data with trend but no seasonality",
    "   • Linear growth/decline patterns",
    "   • Medium-term forecasts\n",
    "✅ Triple Exponential Smoothing (TES/Holt-Winters):",
    "   • Data with trend AND seasonality",
    "   • Regular repeating patterns",
    "   • Long-term forecasts",
    "   • Best for complete time series modeling\n",
    f"💡 For this dataset: TES wins (MAE={tes_mae:.2f})"
]

ax4.text(0.05, 0.95, '\n'.join(decision_text), transform=ax4.transAxes,
        fontsize=11, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.tight_layout()
plt.savefig('exponential_smoothing_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print('\n✅ Exponential Smoothing Methods Compared!')
print(f'\nSimple ES MAE:  {ses_mae:.2f}')
print(f'Double ES MAE:  {des_mae:.2f}')
print(f'Triple ES MAE:  {tes_mae:.2f}')
print(f'\n💡 Best method: {"TES" if tes_mae == min(maes) else "DES" if des_mae == min(maes) else "SES"}')

## 🎯 Real-World Projects

### Post-Silicon Validation Projects

#### 1. **Wafer Yield Forecasting System** 💰 $3M+ Annual Savings
- **Objective:** Forecast weekly yield 12 weeks ahead with RMSE < 2% for capacity planning
- **Data:** 2+ years of weekly yield data with quarterly PM patterns
- **Success Metric:** Accurate PM drop predictions enable proactive capacity allocation
- **Implementation:**
  - Use Holt-Winters multiplicative (PM impact grows with yield level)
  - Seasonal period = 12 weeks (quarterly PM cycle)
  - Validate with rolling window cross-validation (train on 52 weeks, test on 12)
  - Alert capacity planners 4 weeks before PM drop (time to adjust schedules)

#### 2. **Test Time Trend Detector** 💰 $2M+ ATE Optimization
- **Objective:** Detect test time drift (equipment degradation) 2 weeks earlier than current methods
- **Data:** Daily average test time per device (milliseconds), 90 days history
- **Success Metric:** Alert when Holt's forecast exceeds control limits (±3σ)
- **Implementation:**
  - Use Holt's method (trend without seasonality)
  - α=0.3 (responsive), β=0.2 (stable trend)
  - Forecast 14 days ahead, flag if forecast > UCL
  - Business value: Proactive PM reduces unplanned downtime 40%

#### 3. **Multi-Site Production Balancer** 💰 $5M+ Capacity Utilization
- **Objective:** Balance wafer allocation across 3 sites using yield forecasts (±5% accuracy)
- **Data:** Weekly yield per site (Site A: 92%, Site B: 88%, Site C: 85%), 104 weeks history
- **Success Metric:** Minimize total wafer starts while meeting demand (±2% yield accuracy)
- **Implementation:**
  - Fit Holt-Winters for each site (site-specific PM schedules)
  - Forecast 8 weeks ahead for each site
  - Optimizer allocates wafers to site with highest forecasted yield
  - Reforecast weekly (adaptive to process changes)

#### 4. **Equipment Utilization Forecaster** 💰 $1.5M+ Tool Planning
- **Objective:** Forecast ATE utilization 4 weeks ahead (±3% MAPE) for tool purchase decisions
- **Data:** Daily ATE utilization % (70-95%), 180 days history with weekly patterns (lower on weekends)
- **Success Metric:** Forecast drives $20M+ tool purchase decisions with 95% confidence
- **Implementation:**
  - Use Holt-Winters additive (weekly pattern, constant amplitude)
  - Seasonal period = 7 days (weekday/weekend effect)
  - Generate 28-day forecast with 95% confidence intervals
  - Recommend tool purchase if forecast > 85% for 3+ consecutive weeks

---

### General AI/ML Projects

#### 5. **E-Commerce Sales Forecasting Engine** 💰 $10M+ Inventory Optimization
- **Objective:** Forecast daily sales for 500+ SKUs (±10% MAPE) for inventory planning
- **Data:** 2 years of daily sales with yearly seasonality (holidays) + weekly patterns
- **Success Metric:** Reduce stockouts 30%, excess inventory 25%
- **Implementation:**
  - Holt-Winters multiplicative (holiday sales proportional to baseline)
  - Seasonal periods: 7 (weekly) + 365 (yearly) via nested seasonality
  - Forecast 30 days ahead for each SKU
  - Automatically adjust safety stock based on forecast variance

#### 6. **Energy Demand Predictor** 💰 $50M+ Grid Optimization
- **Objective:** Forecast hourly electricity demand 24 hours ahead (±5% accuracy)
- **Data:** 3 years of hourly demand with daily (peak at 6pm) + weekly (lower weekends) patterns
- **Success Metric:** Minimize peak demand charges, optimize generator scheduling
- **Implementation:**
  - Holt-Winters additive (constant daily fluctuations)
  - Seasonal period = 168 hours (weekly cycle)
  - Generate 24-hour forecast every hour (rolling)
  - Alert operators when forecast > 95th percentile (prepare backup generators)

#### 7. **Hotel Occupancy Forecaster** 💰 $5M+ Revenue Management
- **Objective:** Forecast weekly occupancy rate 12 weeks ahead (±8% MAPE) for dynamic pricing
- **Data:** 5 years of weekly occupancy (30-95%) with yearly seasonality (summer peaks)
- **Success Metric:** Increase revenue per available room (RevPAR) 15% via optimal pricing
- **Implementation:**
  - Holt-Winters multiplicative (summer peaks proportional to trend)
  - Seasonal period = 52 weeks (yearly cycle)
  - Forecast 12 weeks ahead for pricing optimization
  - Dynamic pricing: occupancy forecast < 60% → discount 20%, > 80% → premium 30%

#### 8. **Call Center Staffing Optimizer** 💰 $3M+ Labor Cost Reduction
- **Objective:** Forecast hourly call volume 7 days ahead (±15% MAPE) for staff scheduling
- **Data:** 1 year of hourly call volume with daily peaks (10am-2pm) + weekly patterns (Mon/Fri high)
- **Success Metric:** Reduce overtime 25%, maintain service level > 80% (answered in 20s)
- **Implementation:**
  - Holt-Winters additive (constant daily patterns)
  - Seasonal period = 168 hours (weekly cycle)
  - Forecast 168 hours ahead (1 week)
  - Schedule staff = forecast / (avg calls per agent per hour) + 10% buffer

---

## ✅ Key Takeaways

### When to Use Exponential Smoothing

| **Use Case** | **Method** | **Rationale** |
|--------------|-----------|---------------|
| Level only (no trend, no seasonality) | Simple ES | Fast, interpretable (α parameter) |
| Linear trend (no seasonality) | Holt's | Handles improving/declining patterns |
| Trend + constant seasonal amplitude | Holt-Winters Additive | Weekly patterns, fixed fluctuations |
| Trend + growing seasonal amplitude | Holt-Winters Multiplicative | PM drops, holiday sales (proportional) |
| Short-term forecasts (< 12 periods) | Any ES method | ES excels at 1-10 step ahead |
| Long-term forecasts (> 12 periods) | Consider ARIMA/Prophet | ES assumes constant trend/seasonality |

### Advantages ✅
- **No stationarity requirement** (vs ARIMA)
- **Fast computation** (recursive updates)
- **Interpretable parameters** (α, β, γ have clear meanings)
- **Automatic model selection** (AIC/BIC comparison)
- **Built-in seasonality handling** (no feature engineering)

### Limitations ❌
- **Assumes linear trend** (non-linear trends poorly captured)
- **Fixed seasonal patterns** (can't adapt to changing seasonality)
- **No exogenous variables** (can't incorporate external factors like promotions)
- **Forecast uncertainty underestimated** (confidence intervals often too narrow)

### Comparison with Alternatives

| **Method** | **Strengths** | **Weaknesses** |
|------------|---------------|----------------|
| **Exponential Smoothing** | Fast, no stationarity, built-in seasonality | Fixed patterns, no exogenous variables |
| **ARIMA** | Complex autocorrelations, SARIMA for seasonality | Requires stationarity, manual parameter tuning |
| **Prophet** | Automatic seasonality, changepoints, holidays | Slower, opaque forecasting (additive components) |
| **LSTM/Neural Nets** | Non-linear patterns, multivariate | Requires large data, opaque, slow training |

---

## 🚀 Next Steps
- **Notebook 033**: Prophet and modern time series methods
- **Notebook 034**: Advanced forecasting techniques
- **Recommended Practice**: Implement Holt-Winters on real STDF yield data
- **Further Reading**: Hyndman & Athanasopoulos - *Forecasting: Principles and Practice*

## 📊 Summary & Next Steps

**✅ Completed:** Exponential Smoothing methods (SES, Holt, Holt-Winters) with semiconductor applications

**🎯 Key Takeaways:**
- SES for stationary data (power consumption smoothing)
- Holt's method for trend data (yield trending)
- Holt-Winters for seasonal data (quarterly production patterns)
- α, β, γ parameters control smoothing vs responsiveness

**📈 Next Steps:**
- **033: Prophet** - Facebook's modern time series library
- **034: VAR** - Multivariate time series modeling
- Apply exponential smoothing to real production data

**💡 Production Impact:** These forecasting models enable proactive capacity planning, saving $5-15M annually in semiconductor manufacturing.